## A simple model for demand and supply of geolocalized services in a city

### Load libraries and classes

In [ ]:
import numpy as np
import pandas as pd

import os
import sys
nb_dir = os.path.dirname(os.getcwd()) ## TODO: find way to put this into some global settings
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg, istat_kpi, city_settings

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model
from src.models.core import ServiceUnit, ServiceValues, ServiceEvaluator, \
    MappedPositionsFrame, DemandFrame, KPICalculator
from src.models.factories import UnitFactory
from src.models.process_tools import GridMaker, ValuesPlotter, JSONWriter
from src.models.city_items import get_random_pos

In [ ]:
selectedCity = 'Torino'

### Load service data: locations, scales, other info for city

In [ ]:
loaders=UnitFactory.make_loaders_for_city(city_settings.get_city_config(selectedCity))
loaders

In [ ]:
# Initialise with a default lengthscale of 0.5 km
schoolUnits = loaders['Scuole'].load(meanRadius=0.5)

# Initialise with a default lengthscale of 0.3 km
pharmacyUnits = loaders['Farmacie'].load(meanRadius=0.3)

# Initialise with a default lengthscale of 0.3 km
## awful hack to quickly clean tpl file #FIXME
loaders['Fermate TPL']._rawData = loaders['Fermate TPL']._rawData[
    loaders['Fermate TPL']._rawData['route_type'].isin([0,1,3])].reset_index()

tplUnits = loaders['Fermate TPL'].load(meanRadius=0.3)

# Initialise with a default lengthscale of 0.5 km
#libraryUnits = bibliotecheLoader.load(meanRadius=0.8)

### Demand import from Censimento Popolazione e Abitazioni (CPA) 2011

In [ ]:
demandData = DemandFrame.create_from_istat_cpa(selectedCity)

bDemo = False
if bDemo:
    # demo mode, use a portion of the data
    demandTest = DemandFrame(demandData.sample(50).copy(), False)
    testUnits = schoolUnits[::10] + pharmacyUnits[::10] + tplUnits[::30] 
    calculator = KPICalculator(demandTest, testUnits, selectedCity)
else:
    calculator = KPICalculator(demandData, schoolUnits + pharmacyUnits + tplUnits, selectedCity)

### Evaluate the services offer at the demand points and average over neighbourhood

In [ ]:
# compute and plot demand/supply interaction for localized services 
calculator.evaluate_services_at_demand() # this might take a while.
calculator.compute_kpi_for_localized_services()
pass

### Append Istat KPI and export averaged values by Neighbourhood-Service-AgeGroup to JSON

In [ ]:
# compute istat kpi as well
calculator.compute_kpi_for_istat_values()

# write KPI to json output
jsonMaker = JSONWriter(calculator)
jsonMaker.write_all_files_to_default_path()